<a href="https://colab.research.google.com/github/lee-messi/machine-learning/blob/main/cola_grammar_text_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow --quiet
!pip install tensorflow-hub --quiet
!pip install tensorflow-text --quiet
!pip install tensorflow-addons --quiet
!pip install tensorflow-datasets --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 13.7 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow_addons as tfa
import tensorflow_datasets as tfds

tf.get_logger().setLevel('ERROR')
os.environ["TFHUB_MODEL_LOAD_FORMAT"]="UNCOMPRESSED"

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


## Import CoLA (The Corpus of Linguistic Acceptability) Data

CoLA is a collection of sentences that has been annotated for acceptability. The sentences are coded/labeled 1 if the sentence is grammatically acceptable or correct, and they are coded 0 if the sentence is grammatically wrong. You can access the publicly available corpus using the link [here](https://nyu-mll.github.io/CoLA/). First, we are going to import the datasets. The datasets are provided in **.tsv** format which can be imported using the **read_csv()** function.

In [ ]:
train = pd.read_csv('drive/MyDrive/Colab Notebooks/cola-grammar/in_domain_train.tsv',
                    sep = '\t', header = None)
val = pd.read_csv('drive/MyDrive/Colab Notebooks/cola-grammar/in_domain_dev.tsv',
                  sep = '\t', header = None)
test = pd.read_csv('drive/MyDrive/Colab Notebooks/cola-grammar/out_of_domain_dev.tsv',
                   sep = '\t', header = None)

In [ ]:
train.columns = ['misc', 'labels', 'na', 'text']
val.columns = ['misc', 'labels', 'na', 'text']
test.columns = ['misc', 'labels', 'na', 'text']

In [ ]:
'{}{}{}'.format(train.shape, val.shape, test.shape)

'(8551, 4)(527, 4)(516, 4)'

In [ ]:
train = train.groupby('labels').sample(2000)

In [ ]:
def df_to_dataset(dataframe, shuffle = True, batch_size = 32):
  df = dataframe.copy()
  label = df.labels
  df = df.text
  ds = tf.data.Dataset.from_tensor_slices((df, label))
  if shuffle == True:
    ds = ds.shuffle(buffer_size = len(df))
  ds = ds.batch(batch_size)
  ds = ds.prefetch(tf.data.AUTOTUNE)
  return(ds)

In [ ]:
train_ds = df_to_dataset(train)
val_ds = df_to_dataset(val)
test_ds = df_to_dataset(test)

## Classifier Model using Text Embedding

In [ ]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, dtype = tf.string, trainable = True)

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(1, activation = 'sigmoid'))

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss = tf.keras.losses.BinaryCrossentropy(),
              metrics = ['accuracy'])

In [ ]:
history = model.fit(train_ds,
                    validation_data = val_ds,
                    epochs = 10)

Epoch 1/10
125/125 [==============================] - 91s 722ms/step - loss: 0.6781 - accuracy: 0.5700 - val_loss: 0.6932 - val_accuracy: 0.4934
Epoch 2/10
125/125 [==============================] - 90s 723ms/step - loss: 0.6226 - accuracy: 0.6833 - val_loss: 0.6789 - val_accuracy: 0.5617
Epoch 3/10
125/125 [==============================] - 91s 729ms/step - loss: 0.5740 - accuracy: 0.7408 - val_loss: 0.6778 - val_accuracy: 0.5769
Epoch 4/10
125/125 [==============================] - 92s 737ms/step - loss: 0.5244 - accuracy: 0.7680 - val_loss: 0.6875 - val_accuracy: 0.5882
Epoch 5/10
125/125 [==============================] - 90s 717ms/step - loss: 0.4751 - accuracy: 0.7972 - val_loss: 0.6996 - val_accuracy: 0.5977
Epoch 6/10
125/125 [==============================] - 89s 716ms/step - loss: 0.4284 - accuracy: 0.8217 - val_loss: 0.7276 - val_accuracy: 0.5750
Epoch 7/10
125/125 [==============================] - 88s 705ms/step - loss: 0.3862 - accuracy: 0.8365 - val_loss: 0.7643 - val_ac

## Evaluate Model

In [ ]:
model.evaluate(test_ds)

17/17 [==============================] - 3s 124ms/step - loss: 0.8590 - accuracy: 0.4981


[0.8590083718299866, 0.49806201457977295]

The model returned an accuracy of **63.95%**. The model ranks around 15th among the models that have been used to perform the identical task on [Kaggle](https://www.kaggle.com/competitions/cola-in-domain-open-evaluation/leaderboard) (although the leaderboard is quite outdated).